In [ ]:
import os
import tarfile
import requests
import cv2
import glob
import numpy as np
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.image as Image
from sklearn.decomposition import FastICA

In [ ]:
%matplotlib inline

## PCA and ICA

In [ ]:
PATCH_LEN = 8
MAX_PATCHES = 1000
PCA_COMPONENTS = 30
ICA_COMPONENTS = 30
BINS = 15

In [ ]:
def download_data(data_dir="./input/"):

    urls = ["http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/BSR/BSR_bsds500.tgz"]

    for url in urls:
        if not os.path.isdir(data_dir):
            os.makedirs(data_dir)

        file_name = data_dir + "dataset.tgz"
        if os.path.isfile(file_name):
            print("{} downloaded already".format(file_name))
        else:
            print("Downloading '{}' into '{}' file".format(url, file_name))

            data_req = requests.get(url)
            with open(file_name, "wb") as file:
                file.write(data_req.content)

            print("Extracting {} into {}".format(file_name, data_dir))
            if file_name.endswith("tgz"):
                tar = tarfile.open(file_name, "r:")
                tar.extractall(path=data_dir)
                tar.close()
            else:
                print("format unknown")

    print("input data setup success")

In [ ]:
def load_overlapping_patches(path):
    """
    Create MAX_PATCHES overlapping patches from an image
    :param path: Path to an image
    :return: Image patches array
    """

    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    image_patches = extract_patches_2d(image=image, patch_size=(PATCH_LEN, PATCH_LEN), max_patches=MAX_PATCHES)
    image_patches = image_patches.reshape(-1, PATCH_LEN * PATCH_LEN)

    return image_patches


def load_image_patches(path="./input/BSR/BSDS500/data/images/train/"):
    """
    Creates MAX_PATCHES patches from every image in the path and returns it
    :param path: Directory where images are stored
    :return: Image patches of all images in the path
    """

    download_data()
    images_path_list = glob.glob(f"{path}/*.jpg")

    image_patches = []

    for image_path in images_path_list:
        patches = load_overlapping_patches(image_path)
        image_patches.append(patches)

    image_patches = np.vstack(image_patches)

    return image_patches

In [ ]:
def plot_img_grid(images, method, num_images_per_row=5,):

    component = 0
    num_rows = len(images) // num_images_per_row
    f, ax = plt.subplots(num_rows, num_images_per_row)

    for i in range(num_rows):
        for j in range(num_images_per_row):
            component += 1
            ax[i, j].imshow(images[i * num_images_per_row + j])
            ax[i, j].set_title(component)
            ax[i, j].axis("off")

        f.set_figheight(20)
        f.set_figwidth(20)
    
    if not os.path.exists(f"./results/natural_image_{method}/"):
        os.makedirs(f"./results/natural_image_{method}/")

    plt.savefig(f"./results/natural_image_{method}/{MAX_PATCHES}.png")
    plt.show()

In [ ]:
from sklearn.metrics import mutual_info_score

def calc_MI(x, y, bins):
    c_xy = np.histogram2d(x, y, bins)[0]
    mi = mutual_info_score(None, None, contingency=c_xy)
    return mi

In [ ]:
def natural_img_pca(path="./input/BSR/BSDS500/data/images/train/"):

    data = load_image_patches(path)

    image_pca = PCA(n_components=PCA_COMPONENTS)
    image_pca.fit(data)
    pca_weights = []

    for ind, pca_component in enumerate(image_pca.components_, 1):
        image = pca_component.reshape(PATCH_LEN, PATCH_LEN)
        pca_weights.append(image)
    
    #plot_img_grid(pca_weights, method = 'pca')
    transformed_data_x = image_pca.transform(data)[:, 0]
    transformed_data_y = image_pca.transform(data)[:, 1]
    mi = calc_MI(transformed_data_x, transformed_data_y, BINS)

    print(f'Mutual Information for top two principal components in terms of their variance for {MAX_PATCHES} patches in PCA is : {mi}')

In [ ]:
def natural_img_ica(path="./input/BSR/BSDS500/data/images/train/"):
    """
    Calculates the ICA of natural images and plots the weights
    :return: None
    """

    data = load_image_patches(path)

    image_ica = FastICA(n_components=ICA_COMPONENTS)
    image_ica.fit(data)
    ica_weights = []
    ica_weights_var = []

    for ind, ica_component in enumerate(image_ica.components_, 1):
        image = ica_component.reshape(PATCH_LEN, PATCH_LEN)
        ica_weights.append(image)
        ica_weights_var.append(np.var(ica_component))
    
    ica_weights_var = np.array(ica_weights_var)
    ind_max, ind_second_max = np.argsort(ica_weights_var)[-1], np.argsort(ica_weights_var)[-2]

    transformed_data_x = image_ica.transform(data)[:, ind_max]
    transformed_data_y = image_ica.transform(data)[:, ind_second_max]
    mi = calc_MI(transformed_data_x, transformed_data_y, BINS)

    print(f'Mutual Information for top two independent components in terms of their variance for {MAX_PATCHES} patches in ICA is : {mi}')

    #plot_img_grid(ica_weights, method = 'ica')

In [ ]:
def MI_pair_of_nearby_pix(MAX_PATCHES, path="./input/BSR/BSDS500/data/images/train/"):

    data = load_image_patches(path)[1]
    loc1 = 6
    loc2 = 10
    x= np.zeros(data.shape[0])
    y = np.zeros(data.shape[0])
    for i in range(len(data)):
        x[i] = data[loc1]
        y[i] = data[loc2]

    mi = calc_MI(x, y, 20)

    print(f'Mutual Information of a pair of nearby pixels for {MAX_PATCHES} patches per image is : {mi}')

    return



In [ ]:
natural_img_pca()

In [ ]:
MI_pair_of_nearby_pix(MAX_PATCHES)

In [ ]:
natural_img_ica()

## Image Compression

In [ ]:
def YUV_read_frame(filename, size):
    """
    Reads and reconstructs frames from YUV file
    :param filename: Path to the YUV file
    :param size: Dimensions of the frame
    :return: List of frames
    """

    height, width = size
    frame_len = width * height * 3 // 2
    shape = (int(height * 1.5), width)

    frames = []
    with open(filename, "rb") as file:
        while True:
            try:
                raw = file.read(frame_len)
                yuv = np.frombuffer(raw, dtype=np.uint8)
                yuv = yuv.reshape(shape)

                img = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR_I420, 3)
                frames.append(img)
            except:
                break

    return frames


def display_frames(frames):
    """
    Displays a list of frames as a video
    :param frames: List of frames
    :return: None
    """

    for frame in frames:
        cv2.imshow("f", frame)
        cv2.waitKey(40)


def save_frames(frames, path):
    """
    Saves a list of frames in the given path
    :param frames: List of frames
    :param path: Directory where image has to be saved
    :return: None
    """

    os.makedirs(path, exist_ok=True)
    for ind, frame in enumerate(frames, 1):
        cv2.imwrite(f"{path}/{ind:03d}.jpg", frame)

In [ ]:
def load_frames(path):
    """
    Loads all the images in a directory as a list of frames
    :param path: Directory of images
    :return: List of frames
    """

    files = glob.glob(f"{path}/*.jpg")

    files.sort()
    frames = []
    for file in files:
        frame = cv2.imread(file)
        frames.append(frame)

    return frames


def down_sample(frames):
    """
    Downsample a list of frames by a factor of 2
    :param frames: List of frames
    :return: Downsampled frames
    """

    downsampled_frames = []
    for frame in frames:
        downsampled_frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
        downsampled_frames.append(downsampled_frame)

    return downsampled_frames


def up_sample(frames):
    """
    Upsample a list of frames by a factor of 2
    :param frames: List of frames
    :return: Upsampled frames
    """

    upsampled_frames = []
    for frame in frames:
        upsampled_frame = cv2.resize(frame, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        upsampled_frames.append(upsampled_frame)

    return upsampled_frames


def plot_comparision(first_data, second_data, xlabel, ylabel, title, method):
    """
    Plots the two MSE vs Bitrate graph for comparision
    :param firstMSE: First dictionary with Bitrate - MSE as key - value pairs
    :param secondMSE: Second dictionary with Bitrate - MSE as key - value pairs
    :return: None
    """

    plt.figure()
    plt.plot(first_data.keys(), first_data.values(), label="Compress - Decompress")
    plt.plot(second_data.keys(), second_data.values(), label="Downsample - Compress - Decompress - Upsample")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.savefig(f"./results/image_compression_and_resolution/{method}/comparision.png")
    plt.show()

In [ ]:
HEIGHT = 432
WIDTH = 768

LEVEL = 3
SIGMA = 1.5
KERNEL = (3, 3)

BITRATE_START = 100
BITRATE_END = 1001
BITRATE_STEP = 100

In [ ]:
def compress(filename):
    """
    Compress the specified file with H.264 compression and saves it in the results directory
    (One I frame and rest P frames)
    :param filename: YUV file to be compressed
    :return: None
    """

    frames = YUV_read_frame(filename, (HEIGHT, WIDTH))
    # display_frames(frames)

    path = f".{filename.split('.')[1]}"
    save_frames(frames, path)
    total_frames = len(os.listdir(path))

    for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):
        os.system(
            f"ffmpeg -hide_banner -loglevel panic -i {path}/%03d.jpg -b:v {bitrate}k -c:v libx264 "
            f"-x264-params keyint=300:min-keyint=300:no-scenecut=1:bframes=0 -c:a copy -y "
            f"./results/image_compression_and_resolution/I1/compress/{bitrate}.h264"
        )

        frame_details = os.popen(
            f"ffprobe -hide_banner -loglevel panic -show_frames "
            f"./results/image_compression_and_resolution/I1/compress/{bitrate}.h264 | grep pict_type"
        )

        I_frame_count = 0
        P_Frame_count = 0
        for line in frame_details:
            type = line.rstrip().split("=")[-1]

            if type == "I":
                I_frame_count += 1
            else:
                P_Frame_count += 1

        print(f"I Frame Count: {I_frame_count}, P Frame Count: {P_Frame_count}, Total Frames: {total_frames}")


def decompress():
    """
    Decompresses a H.264 file into frames and saves it in a directory
    :return: None
    """

    """
    Reads a H.264 file, extracts, and saves the image frames to a directory
    '-qscale:v ' specifies the quality of the JPEG frames (Max quality: 2)
    'hide-banner' and 'loglevel panic' options are for setting the logging to minimum
    """
    for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):
        os.makedirs(f"./results/image_compression_and_resolution/I1/compress/{bitrate}/", exist_ok=True)
        os.system(
            f"ffmpeg -hide_banner -loglevel panic -i "
            f"./results/image_compression_and_resolution/I1/compress/{bitrate}.h264"
            f" -qscale:v 2 ./results/image_compression_and_resolution/I1/compress/{bitrate}/%03d.jpg"
        )

In [ ]:
def downsample_compress(filename):
    """
    Downsamples (spatially by 2) and compress the specified file with H.264 compression and saves it in the
    results directory (One I frame and rest P frames)
    :param filename: YUV file to be compressed
    :return: None
    """

    frames = YUV_read_frame(filename, (HEIGHT, WIDTH))
    # display_frames(frames)

    frames = down_sample(frames)
    path = f".{filename.split('.')[1]}_downsampled"
    save_frames(frames, path)
    total_frames = len(os.listdir(path))

    for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):
        os.system(
            f"ffmpeg -hide_banner -loglevel panic -i {path}/%03d.jpg -b:v {bitrate}k -c:v libx264 "
            f"-x264-params keyint=300:min-keyint=300:no-scenecut=1:bframes=0 -c:a copy -y "
            f"./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}.h264"
        )

        frame_details = os.popen(
            f"ffprobe -hide_banner -loglevel panic -show_frames "
            f"./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}.h264 | grep pict_type"
        )

        I_frame_count = 0
        P_Frame_count = 0
        for line in frame_details:
            type = line.rstrip().split("=")[-1]

            if type == "I":
                I_frame_count += 1
            else:
                P_Frame_count += 1

        print(f"I Frame Count: {I_frame_count}, P Frame Count: {P_Frame_count}, Total Frames: {total_frames}")


def decompress_upsample():
    """
    Decompresses a H.264 file into frames and saves it in a directory
    Frames are upsampled and saved in the upsampled directory
    :return: None
    """

    for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):
        os.makedirs(
            f"./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}/", exist_ok=True
        )
        os.system(
            f"ffmpeg -hide_banner -loglevel panic -i "
            f"./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}.h264 "
            f"-qscale:v 2 ./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}/%03d.jpg"
        )

    for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):
        frames = load_frames(f"./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}/")
        frames = up_sample(frames)
        save_frames(
            frames, f"./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}_upsampled/"
        )

In [ ]:
# Compress - Decompress
print("Compressing to H.264 Video")
compress("./input/bs1_25fps.yuv")

print("Decompressing from H.264 Video")
decompress()

# Downsample - Compress - Decompress - Upsample
print("Downsampling and Compressing to H.264")
downsample_compress("./input/bs1_25fps.yuv")

print("Decompressing from H.264 and upsampling")
decompress_upsample()

In [ ]:
def compress(filename):
    """
    Compress the specified file with H.264 compression and saves it in the results directory
    (all I frames)
    :param filename: YUV file to be compressed
    :return: None
    """

    frames = YUV_read_frame(filename, (HEIGHT, WIDTH))
    # display_frames(frames)

    path = f".{filename.split('.')[1]}"
    save_frames(frames, path)
    total_frames = len(os.listdir(path))

    for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):
        os.system(
            f"ffmpeg -hide_banner -loglevel panic -i {path}/%03d.jpg -b:v {bitrate}k -c:v libx264 "
            f"-x264-params keyint=1:min-keyint=1:no-scenecut=1:bframes=0 -c:a copy -y "
            f"./results/image_compression_and_resolution/I/compress/{bitrate}.h264"
        )

        frame_details = os.popen(
            f"ffprobe -hide_banner -loglevel panic -show_frames "
            f"./results/image_compression_and_resolution/I/compress/{bitrate}.h264 | grep pict_type"
        )

        I_frame_count = 0
        P_Frame_count = 0
        for line in frame_details:
            type = line.rstrip().split("=")[-1]

            if type == "I":
                I_frame_count += 1
            else:
                P_Frame_count += 1

        print(f"I Frame Count: {I_frame_count}, P Frame Count: {P_Frame_count}, Total Frames: {total_frames}")


def decompress():
    """
    Decompresses a H.264 file into frames and saves it in a directory
    :return: None
    """

    """
    Reads a H.264 file, extracts, and saves the image frames to a directory
    '-qscale:v ' specifies the quality of the JPEG frames (Max quality: 2)
    'hide-banner' and 'loglevel panic' options are for setting the logging to minimum
    """
    for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):
        os.makedirs(f"./results/image_compression_and_resolution/I/compress/{bitrate}/", exist_ok=True)
        os.system(
            f"ffmpeg -hide_banner -loglevel panic -i "
            f"./results/image_compression_and_resolution/I/compress/{bitrate}.h264"
            f" -qscale:v 2 ./results/image_compression_and_resolution/I/compress/{bitrate}/%03d.jpg"
        )

In [ ]:
def downsample_compress(filename):
    """
    Downsamples (spatially by 2) and compress the specified file with H.264 compression and saves it in the
    results directory (all I frames)
    :param filename: YUV file to be compressed
    :return: None
    """

    frames = YUV_read_frame(filename, (HEIGHT, WIDTH))
    # display_frames(frames)

    frames = down_sample(frames)
    path = f".{filename.split('.')[1]}_downsampled"
    save_frames(frames, path)
    total_frames = len(os.listdir(path))

    for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):
        os.system(
            f"ffmpeg -hide_banner -loglevel panic -i {path}/%03d.jpg -b:v {bitrate}k -c:v libx264 "
            f"-x264-params keyint=1:min-keyint=1:no-scenecut=1:bframes=0 -c:a copy -y "
            f"./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}.h264"
        )

        # Check the frame types and print stats for the compressed file
        frame_details = os.popen(
            f"ffprobe -hide_banner -loglevel panic -show_frames "
            f"./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}.h264 | grep pict_type"
        )

        I_frame_count = 0
        P_Frame_count = 0
        for line in frame_details:
            type = line.rstrip().split("=")[-1]

            if type == "I":
                I_frame_count += 1
            else:
                P_Frame_count += 1

        print(f"I Frame Count: {I_frame_count}, P Frame Count: {P_Frame_count}, Total Frames: {total_frames}")


def decompress_upsample():
    """
    Decompresses a H.264 file into frames and saves it in a directory
    Frames are upsampled and saved in the upsampled directory
    :return: None
    """

    for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):
        os.makedirs(
            f"./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}/", exist_ok=True
        )
        os.system(
            f"ffmpeg -hide_banner -loglevel panic -i "
            f"./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}.h264 "
            f"-qscale:v 2 ./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}/%03d.jpg"
        )

    for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):
        frames = load_frames(f"./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}/")
        frames = up_sample(frames)
        save_frames(
            frames, f"./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}_upsampled/"
        )

In [ ]:
# Compress - Decompress
print("Compressing to H.264 Video")
compress("./input/bs1_25fps.yuv")

print("Decompressing from H.264 Video")
decompress()

# Downsample - Compress - Decompress - Upsample
print("Downsampling and Compressing to H.264")
downsample_compress("./input/bs1_25fps.yuv")

print("Decompressing from H.264 and upsampling")
decompress_upsample()

In [ ]:
psnrI1_dict = {}
vmafI1_dict = {}
psnr_downI1_dict={}
vmaf_downI1_dict={}

"""
Reads a H.264 file, extracts, and saves the image frames to a directory
'-qscale:v ' specifies the quality of the JPEG frames (Max quality: 2)
'hide-banner' and 'loglevel panic' options are for setting the logging to minimum
"""
for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):

    os.system(
        f"FFREPORT=file=./results/image_compression_and_resolution/I1/compress/{bitrate}/{bitrate}_psnr.log:level=32 ffmpeg -hide_banner -loglevel panic "
        f"-i /Video-Compression-and-PCA-master/input/bs1_25fps/%03d.jpg"
        f" -i /Video-Compression-and-PCA-master/results/image_compression_and_resolution/I1/compress/100/%03d.jpg "
        f"-filter_complex psnr -f null -")

    os.system(
        f"FFREPORT=file=./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}/{bitrate}_psnr.log:level=32 ffmpeg -hide_banner -loglevel panic "
        f"-i /Video-Compression-and-PCA-master/input/bs1_25fps/%03d.jpg"
        f" -i /Video-Compression-and-PCA-master/results/image_compression_and_resolution/I1/downsample_compress_upsample/100/%03d.jpg "
        f"-filter_complex psnr -f null -")

    os.system(
        f"FFREPORT=file=./results/image_compression_and_resolution/I1/compress/{bitrate}/{bitrate}_vmaf.log:level=32 ffmpeg -hide_banner -loglevel panic "
        f"-i /Video-Compression-and-PCA-master/input/bs1_25fps/%03d.jpg"
        f"-i /Video-Compression-and-PCA-master/results/image_compression_and_resolution/I1/compress/{bitrate}/%03d.jpg "
        f"-filter_complex libvmaf -f null -")

    os.system(
        f"FFREPORT=file=./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}/{bitrate}_vmaf.log:level=32 ffmpeg -hide_banner -loglevel panic "
        f"-i /Video-Compression-and-PCA-master/input/bs1_25fps/%03d.jpg"
        f"-i /Video-Compression-and-PCA-master/results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}/%03d.jpg "
        f"-filter_complex libvmaf -f null -")

    val = float(
        list(os.popen(f"cat ./results/image_compression_and_resolution/I1/compress/{bitrate}/{bitrate}_psnr.log | grep PSNR"))[-1].strip().split(
            " ")[-3].split(":")[-1])
    val_vmaf_down = float(
        list(os.popen(f"cat ./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}/{bitrate}_vmaf.log | grep VMAF"))[-1].strip().split(
            " ")[-3].split(":")[-1])

    val_down = float(
        list(os.popen(f"cat ./results/image_compression_and_resolution/I1/downsample_compress_upsample/{bitrate}/{bitrate}_psnr.log | grep PSNR"))[-1].strip().split(
            " ")[-3].split(":")[-1])
    val_vmaf = float(
        list(os.popen(f"cat ./results/image_compression_and_resolution/I1/compress/{bitrate}/{bitrate}_vmaf.log | grep VMAF"))[-1].strip().split(
            " ")[-3].split(":")[-1])

    psnrI1_dict[bitrate] = val
    vmafI1_dict[bitrate] = val_vmaf
    psnr_downI1_dict[bitrate] = val_down
    vmaf_downI1_dict[bitrate] = val_vmaf_down

In [ ]:
psnrI_dict = {}
vmafI_dict = {}
psnr_downI_dict={}
vmaf_downI_dict={}

"""
Reads a H.264 file, extracts, and saves the image frames to a directory
'-qscale:v ' specifies the quality of the JPEG frames (Max quality: 2)
'hide-banner' and 'loglevel panic' options are for setting the logging to minimum
"""
for bitrate in range(BITRATE_START, BITRATE_END, BITRATE_STEP):

    os.system(
        f"FFREPORT=file=./results/image_compression_and_resolution/I/compress/{bitrate}/{bitrate}_psnr.log:level=32 ffmpeg -hide_banner -loglevel panic "
        f"-i /Video-Compression-and-PCA-master/input/bs1_25fps/%03d.jpg"
        f" -i /Video-Compression-and-PCA-master/results/image_compression_and_resolution/I/compress/100/%03d.jpg "
        f"-filter_complex psnr -f null -")

    os.system(
        f"FFREPORT=file=./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}/{bitrate}_psnr.log:level=32 ffmpeg -hide_banner -loglevel panic "
        f"-i /Video-Compression-and-PCA-master/input/bs1_25fps/%03d.jpg"
        f" -i /Video-Compression-and-PCA-master/results/image_compression_and_resolution/I/downsample_compress_upsample/100/%03d.jpg "
        f"-filter_complex psnr -f null -")

    os.system(
        f"FFREPORT=file=./results/image_compression_and_resolution/I/compress/{bitrate}/{bitrate}_vmaf.log:level=32 ffmpeg -hide_banner -loglevel panic "
        f"-i /Video-Compression-and-PCA-master/input/bs1_25fps/%03d.jpg"
        f"-i /Video-Compression-and-PCA-master/results/image_compression_and_resolution/I/compress/{bitrate}/%03d.jpg "
        f"-filter_complex libvmaf -f null -")

    os.system(
        f"FFREPORT=file=./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}/{bitrate}_vmaf.log:level=32 ffmpeg -hide_banner -loglevel panic "
        f"-i /Video-Compression-and-PCA-master/input/bs1_25fps/%03d.jpg"
        f"-i /Video-Compression-and-PCA-master/results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}/%03d.jpg "
        f"-filter_complex libvmaf -f null -")

    val = float(
        list(os.popen(f"cat ./results/image_compression_and_resolution/I/compress/{bitrate}/{bitrate}_psnr.log | grep PSNR"))[-1].strip().split(
            " ")[-3].split(":")[-1])
    val_vmaf_down = float(
        list(os.popen(f"cat ./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}/{bitrate}_vmaf.log | grep VMAF"))[-1].strip().split(
            " ")[-3].split(":")[-1])

    val_down = float(
        list(os.popen(f"cat ./results/image_compression_and_resolution/I/downsample_compress_upsample/{bitrate}/{bitrate}_psnr.log | grep PSNR"))[-1].strip().split(
            " ")[-3].split(":")[-1])
    val_vmaf = float(
        list(os.popen(f"cat ./results/image_compression_and_resolution/I/compress/{bitrate}/{bitrate}_vmaf.log | grep VMAF"))[-1].strip().split(
            " ")[-3].split(":")[-1])

    psnrI_dict[bitrate] = val
    vmafI_dict[bitrate] = val_vmaf
    psnr_downI_dict[bitrate] = val_down
    vmaf_downI_dict[bitrate] = val_vmaf_down

In [ ]:
print("Case 1: one I frame rest P frames")
plot_comparision(psnrI1_dict, psnr_downI1_dict, "Bitrate in Kbps", "PSNR", "PSNR vs Bitrate", "I1")
plot_comparision(vmafI1_dict, vmaf_downI1_dict, "Bitrate in Kbps", "VMAF", "VMAF vs Bitrate", "I1")

print("Case 2: all I frames")
plot_comparision(psnrI_dict, psnr_downI_dict, "Bitrate in Kbps", "PSNR", "PSNR vs Bitrate", "I")
plot_comparision(vmafI_dict, vmaf_downI_dict, "Bitrate in Kbps", "VMAF", "VMAF vs Bitrate", "I")